In [3]:
import json
import os


for model_name in ["Meta-Llama-3-70B-Instruct", 'Qwen2.5-VL-32B-Instruct']:
    for dataset in ['virtualhome', 'behavior']:
        print(f"Model: {model_name}, Dataset: {dataset}")
        all_outputs = []
        for type in ['goal_interpretation', 'action_sequencing', 'subgoal_decomposition', 'transition_modeling']:
            if dataset  == 'virtualhome' or type == 'transition_modeling':
                path = f'output/{dataset}/evaluate_results/{type}/{model_name}/summary.json'
            else:
                path = f"output/{dataset}/evaluate_results/{type}/summary/{model_name}_performance_scores.json"
                path2 = f'output/{dataset}/evaluate_results/{type}/summary/{model_name}_outputs.json'
                path3 = f'output/{dataset}/evaluate_results/{type}/summary/{model_name}.json'
                
            if os.path.exists(path) or os.path.exists(path2) or os.path.exists(path3):
                if os.path.exists(path):
                    path = path
                elif os.path.exists(path2):
                    path = path2
                else:
                    path = path3
                with open(path) as f:
                    data = json.load(f)
                if type in ['action_sequencing', 'subgoal_decomposition']:
                    task_success_rate = data['goal_evaluation']['task_success_rate']
                    if type == 'subgoal_decomposition' or dataset == 'behavior':
                        task_success_rate *= 100
                    task_success_rate = round(task_success_rate, 2)
                    execution_success_rate = data['trajectory_evaluation']['execution_success_rate']
                    if type == 'subgoal_decomposition' or dataset == 'behavior':
                        execution_success_rate *= 100
                    execution_success_rate = round(execution_success_rate, 2)
                    all_outputs.append(f"{task_success_rate}\t{execution_success_rate}")
                elif type == 'transition_modeling':
                    f1 = data["overall"]["f1"]
                    f1 = round(f1, 2)
                    planner_success_rate = data["overall"]["planner_success_rate"]
                    planner_success_rate = round(planner_success_rate, 2)
                    all_outputs.append(f"{f1}\t{planner_success_rate}")
                elif type == 'goal_interpretation':
                    if "all_f1" in data:
                        all_f1 = data['all_f1']
                        all_f1 = round(all_f1, 2)
                    else:
                        all_f1 = data['overall']['overall_confusion_metrics']['f1_score']*100
                        all_f1 = round(all_f1, 2)
                    all_outputs.append(all_f1)
            else:
                if type == 'goal_interpretation':
                    all_outputs.append(-1)
                else:
                    all_outputs.append('-1\t-1')

        print('\t'.join([str(i) for i in all_outputs]))

Model: Meta-Llama-3-70B-Instruct, Dataset: virtualhome
31.58	58.69	63.3	80.77	87.57	32.26	10.14
Model: Meta-Llama-3-70B-Instruct, Dataset: behavior
74.07	28.0	35.0	23.0	29.0	53.7	68.0
Model: Qwen2.5-VL-32B-Instruct, Dataset: virtualhome
25.64	52.13	57.7	73.96	79.59	8.31	31.53
Model: Qwen2.5-VL-32B-Instruct, Dataset: behavior
72.43	30.0	35.0	28.0	34.0	46.87	22.0
